In [1]:
import os

from main import STAGE_NAME

In [2]:
%pwd

'C:\\Users\\RameezRassdeen\\Documents\\wine-quality-prediction-webapp\\notebooks'

In [3]:
os.chdir('..')

In [4]:
import pandas as pd

data = pd.read_csv('artifacts/data_ingestion/wine_quality_data/winequality-red.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    """
    Configuration for data validation.
    """
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [7]:
from wine_quality_prediction.constants import *
from wine_quality_prediction.utils.common import read_yaml, create_directory

In [8]:
class ConfigurationManager:
    def __init__(self,
            config_file_path: Path = CONFIG_FILE_PATH,
            params_file_path: Path = PARAMS_FILE_PATH,
            schema_file_path: Path = SCHEMA_FILE_PATH):

            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)
            self.schema = read_yaml(schema_file_path)

            create_directory([self.config.artifacts_root], verbose=True)

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directory([config.root_dir], verbose=True)

        data_validation_config_ = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema= schema
        )

        return data_validation_config_

In [9]:
from wine_quality_prediction import logger

In [10]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            wine_data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(wine_data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as file:
                        file.write(f"Validation Status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as file:
                        file.write(f"Validation Status: {validation_status}")
            return validation_status

        except Exception as e:
            logger.exception(e)
            raise e

In [11]:
STAGING_NAME = "STAGE 02: DATA VALIDATION"

try:
    logger.info(f">>>>> Starting {STAGING_NAME} <<<<<")
    config_manager = ConfigurationManager()
    data_validation_config = config_manager.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
    logger.info(f">>>>> Completed {STAGING_NAME} <<<<<")

except Exception as e:
    logger.exception(e)
    raise e

[2025-01-22 21:37:14,949 - wine_quality_prediction - INFO - >>>>> Starting STAGE 02: DATA VALIDATION <<<<<]
[2025-01-22 21:37:14,953 - wine_quality_prediction - INFO - YAML file loaded successfully from the path: config\config.yaml]
[2025-01-22 21:37:14,955 - wine_quality_prediction - INFO - YAML file loaded successfully from the path: params.yaml]
[2025-01-22 21:37:14,957 - wine_quality_prediction - INFO - YAML file loaded successfully from the path: schema.yaml]
[2025-01-22 21:37:14,959 - wine_quality_prediction - INFO - Directory already exists at the path: artifacts]
[2025-01-22 21:37:14,961 - wine_quality_prediction - INFO - Directory created at the path: artifacts/data_validation]
[2025-01-22 21:37:14,972 - wine_quality_prediction - INFO - >>>>> Completed STAGE 02: DATA VALIDATION <<<<<]
